In [271]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import unidecode
import spacy

In [287]:
top = pd.read_csv('top200.csv')

bottom = pd.read_csv('bottom200.csv')

In [288]:
top['lyrics']

0      [Paroles de "DIE"] [Intro] La mala est gangx E...
1      [Paroles de "Balader" ft. Niska] [Intro :  Soo...
2      [Paroles de "FADE UP" ft. Hamza & SCH] [Intro ...
3      [Chorus] I'm good yeah I'm feelin' alright Bab...
4      [Paroles de "Carolina" ft. Ninho] [Refrain] Ca...
                             ...                        
263    [Verse 1] And all I am is a man I want the wor...
264    [Paroles de "Dis-moi que tu m'aimes"] [Intro :...
265    [Paroles de "Dernier Etage" ft. Ninho] [Intro ...
266    [Paroles de "Mentalité"] [Intro] Yah [Couplet ...
267    [Paroles de "Flash"] [Couplet 1] J'me souviens...
Name: lyrics, Length: 268, dtype: object

In [289]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

In [290]:
def stop_words_french(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if word not in stopwords.words('french') and len(word)>1]
    return text

def stop_words_english(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stopwords.words('english') and len(word)>1]
    return text

In [291]:
def add_lyrics_features(df):
    df['lyrics']=df['lyrics'].dropna()
    df['lyrics_nb_max'] = 0
    df['lyrics_nb_moy'] = 0
    df['lyrics_nb_std'] = 0
    i = 0
    while i < len(df['lyrics']):
        
        df['lyrics'][i] = str(df['lyrics'][i])

        if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
            df['lyrics'][i] = unidecode.unidecode(df['lyrics'][i])
            df['lyrics'][i] = re.sub(r'\[.*?\]', '', df['lyrics'][i])
            carac = ["'", "-", "!", ".", "?", "(", ")"]
            for c in carac:
                df['lyrics'][i] = df['lyrics'][i].replace(c, ' ')
            df['lyrics'][i] = df['lyrics'][i].lower()
            df['lyrics'][i] = df['lyrics'][i].split(' ')
            #lyrics[i] = lemmatization(nlp_fr, lyrics[i])
            # lyrics[i] = lemmatization(nlp_en, lyrics[i])
            df['lyrics'][i] = stop_words_english(df['lyrics'][i])
            df['lyrics'][i] = stop_words_french(df['lyrics'][i])

            if (df['lyrics'][i] != 'nan') and (len(df['lyrics'][i]) > 0):
                d = {j:df['lyrics'][i].count(j) for j in df['lyrics'][i]}

                max = np.array(list(d.values())).max()
                mean = np.array(list(d.values())).mean()
                std = np.array(list(d.values())).std()

                df['lyrics_nb_max'][i] = max
                df['lyrics_nb_moy'][i] = mean
                df['lyrics_nb_std'][i] = std

        i +=1
    return df

In [293]:
top = add_lyrics_features(top)
top.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,lyrics,id,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
0,DIE,Gazo,"drill francais, rap francais",1,0.695,0.630,8,-7.160,0,0.0350,...,0.118,0.550,130.968,240413,4,"[mala, gangx, eh, bang, bang, eh, grr, grr, ba...",3D29kjUyWxsT3jUUTtARVQ,4,1.800000,0.900000
1,Balader,Soolking,"francoton, french hip hop, pop urbaine, rap al...",2,0.603,0.802,4,-5.089,0,0.1110,...,0.124,0.787,130.121,176120,4,"[ah, tou, tou, tou, touh, bebe, fais, sous, re...",6PrB4KpvuoYWtwjPC5a44S,11,2.009259,1.863367
2,FADE UP,ZEG P,NaN,3,0.758,0.599,4,-5.990,0,0.0877,...,0.129,0.557,129.960,219188,4,"[zeg, track, bitch, passe, toute, night, compt...",4ZpIuzx91EAPK3VimONbfB,22,2.258621,3.088133
3,I'm Good (Blue),David Guetta,"big room, dance pop, edm, pop, pop dance",4,0.561,0.965,7,-3.673,0,0.0343,...,0.371,0.304,128.040,175238,4,"[good, yeah, feelin, alright, baby, best, fuck...",4uUG5RXrOk84mYEfFvj3cK,13,3.100000,3.176476
4,Carolina,Ninho,"french hip hop, pop urbaine",5,0.791,0.904,11,-2.830,0,0.0956,...,0.106,0.361,100.077,195459,4,"[carolina, caro, pete, saoul, pete, saoul, deh...",22IP2WKjuGYQ19JAz5GpAv,9,2.277108,1.652404


In [294]:
bottom = add_lyrics_features(bottom)
bottom.head()

,title,artist,genre,rank,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,duration_ms,time_signature,lyrics,id,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
0,IMPACT,Gazo,"drill francais, rap francais",1,0.767,0.673,8.0,-5.843,0.0,0.2280,...,0.0995,0.314,140.075,186400,4.0,"[gang, gang, gang, gang, ethan, jackson, prod,...",1hlUHJgbdFdvZvG64a3W1g,12,1.910256,1.529085
1,GRA GRA BOOM (feat. Skread),Gazo,"drill francais, rap francais",2,0.668,0.773,5.0,-5.841,1.0,0.0648,...,0.1150,0.492,143.000,208413,4.0,"[mala, gangx, attends, die, venir, dire, etais...",3aOzpOcVaAXCb2bkabANde,13,2.278846,2.026064
2,HENNESSY,Gazo,"drill francais, rap francais",3,0.842,0.615,10.0,-8.325,1.0,0.2050,...,0.0922,0.204,144.128,217960,4.0,"[gangx, hey, s/o, flem, anh, ouh, hey, hey, an...",6fKYXAnTD96mFM9nAGaC7V,11,1.593220,1.485736
3,Zabouza,Soolking,"francoton, french hip hop, pop urbaine, rap al...",4,0.823,0.753,5.0,-11.994,0.0,0.1030,...,0.3350,0.386,95.234,144227,3.0,"[ah, ah, ouh, oui, ouh, yeah, ouh, yeah, veule...",1oZQCHtqXw9XSznV4bRc4E,6,1.239130,0.707664
4,Good Summer,Soolking,"francoton, french hip hop, pop urbaine, rap al...",5,0.821,0.710,7.0,-8.445,0.0,0.1240,...,0.1960,0.545,93.998,155293,4.0,"[ouais, ouais, fait, deuil, sentiments, fait, ...",7mofgPCHJ1HSUhb834WuEH,12,1.716312,1.512718


In [295]:
top.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
count,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000,268.000000
mean,134.500000,0.708556,0.648933,5.641791,-6.943571,0.395522,0.148654,0.301629,0.003639,0.147038,0.525579,124.411944,194705.294776,3.962687,12.201493,1.729868,1.659084
std,77.509139,0.116823,0.140500,3.541943,2.157361,0.489877,0.124393,0.223156,0.027600,0.092010,0.218318,26.045026,40720.626471,0.284604,10.361467,0.485877,1.096319
min,1.000000,0.351000,0.277000,0.000000,-15.248000,0.000000,0.025800,0.001460,0.000000,0.028800,0.037400,59.974000,74571.000000,1.000000,0.000000,0.000000,0.000000
25%,67.750000,0.640000,0.545750,2.000000,-8.284250,0.000000,0.053800,0.119250,0.000000,0.094000,0.361750,104.150750,167164.000000,4.000000,6.000000,1.435166,0.997720
50%,134.500000,0.721000,0.660500,6.000000,-6.588000,0.000000,0.089200,0.263500,0.000000,0.115500,0.523000,126.820000,190313.000000,4.000000,9.000000,1.679695,1.396053
75%,201.250000,0.787500,0.744500,9.000000,-5.543750,1.000000,0.223250,0.440250,0.000057,0.162250,0.706000,140.089000,216150.750000,4.000000,14.000000,1.918848,1.871763
max,268.000000,0.959000,0.965000,11.000000,-2.046000,1.000000,0.724000,0.947000,0.402000,0.543000,0.972000,191.979000,356347.000000,5.000000,85.000000,3.906977,7.398106


In [296]:
bottom.describe()

,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics_nb_max,lyrics_nb_moy,lyrics_nb_std
count,327.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,327.000000,325.000000,327.000000,327.000000,327.000000
mean,164.000000,0.669455,0.587543,5.436923,-8.348022,0.418462,0.185727,0.357935,0.037148,0.165962,0.482587,119.926200,193915.247706,3.892308,11.605505,1.649148,1.649092
std,94.540996,0.168221,0.189838,3.484515,3.709453,0.494067,0.175646,0.270309,0.165027,0.125479,0.233716,29.638944,62115.156206,0.518875,10.306962,0.688095,1.373809
min,1.000000,0.000000,0.018400,0.000000,-27.194000,0.000000,0.000000,0.000321,0.000000,0.000000,0.000000,0.000000,7156.000000,0.000000,0.000000,0.000000,0.000000
25%,82.500000,0.570000,0.486000,2.000000,-9.355000,0.000000,0.052600,0.129000,0.000000,0.096000,0.316000,99.988000,163708.500000,4.000000,6.000000,1.349808,0.842971
50%,164.000000,0.706000,0.619000,5.000000,-7.425000,0.000000,0.126000,0.312000,0.000000,0.114000,0.498000,121.981000,191828.000000,4.000000,9.000000,1.585586,1.321536
75%,245.500000,0.791000,0.721000,8.000000,-6.142000,1.000000,0.271000,0.535000,0.000092,0.187000,0.659000,138.084000,223006.500000,4.000000,14.000000,1.922213,2.060895
max,327.000000,0.962000,0.945000,11.000000,-2.021000,1.000000,0.951000,0.985000,0.948000,0.848000,0.972000,200.578000,442250.000000,5.000000,69.000000,5.000000,10.201351


In [ ]:
def lemmatization(nlp, texte):
    i = 0
    for mot in texte:
        doc = nlp(mot)
        for token in doc:
            texte[i] = token.lemma_.lower()
            
        i += 1
            
    return texte

In [ ]:
nlp_fr = spacy.load('fr_core_news_md')
nlp_en = spacy.load('en_core_web_md')

In [ ]:
i = 0
while i < len(top['lyrics']):
    if len(top['lyrics'][i]) > 0:
        top['lyrics'][i] = lemmatization(nlp_fr, top['lyrics'][i])
        top['lyrics'][i] = lemmatization(nlp_en, top['lyrics'][i])
    i +=1